In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path

In [2]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)

2638744/2638744 [==============================] - 1s 1us/step


In [4]:
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text(encoding="utf-8")

In [5]:
text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [6]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?


In [7]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [9]:
text_vec_layer_en.get_vocabulary()[:10]
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [10]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [11]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [12]:
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True) # mask_zero=True 会把输入 0 视为 padding，并自动生成布尔 mask，使后续层忽略这些 padding 位置，从而避免无意义的 padding 信息污染模型。
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

在2017年一篇开创性的论文 (https://arxiv.org/abs/1706.03762) 中，一组Google研究人员提出“注意力就是你所需要的”。他们创建了一种被称为Transformer的架构，通过注意力机制（加上嵌入层、密集层、归一化层和其他一些要素）极大地提高了机器翻译的水平，而没有使用任何循环层或卷积层。因为该模型不是循环神经网络，所以不像RNN那样容易受到梯度消失或梯度爆炸问题的影响，它可以在较少的步骤中进行训练，更容易在多个GPU之间并行处理，也可以更好地捕捉RNN无法捕捉的长期模式。下图展示了最初的Transformer架构。简而言之，图左侧部分是编码器，右侧部分是解码器。每个嵌入层输出一个形状为［批量大小，序列长度，嵌入大小］的三维张量。然后，在流经Transformer时逐渐转换张量，但张量形状保持不变。如果使用Transformer进行NMT，则在训练期间必须将待翻译句子馈送到编码器，并将相应的目标语言翻译（每个句子开头都插入一个额外的SOS词元）馈送到解码器。在推理时，必须多次调用Transformer，一次一个词地生成翻译，并在每次调用将部分翻译馈送到解码器，就像在translate()函数中所做的那样。编码器的作用是逐渐转换输入——英语句子的单词表示——直到每个单词的表示完美地捕捉到单词在上下文中的含义。例如，如果将句子“I like soccer”馈送给编码器，那么like一词将从一个相当模糊的表示开始，因为这个词在不同的上下文中（如“I like soccer”和“It’s like that”）可能有不同的含义。但是经过编码器处理之后，单词的表示应该捕捉到在给定句子中like的正确含义（即喜欢）以及翻译所需的任何其他信息（例如，它是一个动词）。

<img alt="论文里的Transformer架构" height="1200" src="./images/transformer/p1.png" width="400"/>

解码器的作用是逐步将翻译句子中每个单词的表示转换为下一个单词的表示。例如，如果要翻译的句子是“I like soccer”，解码器的输入句子是“＜SOS＞me gusta el fútbol”，那么经过解码器后，el这个单词的表示将被转换为fútbol的表示。同样，fútbol的表示将被转换为EOS词元的表示。经过解码器后，每个单词表示都通过一个具有softmax激活函数的最终Dense层，希望该层输出下一个正确单词对应的高概率和所有其他单词对应的低概率。预测的句子应该是“me gusta el fútbol＜EOS＞”。

In [21]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

结构图的详细情况：

- 首先，请注意编码器和解码器都包含了堆叠N次的模块。在论文中，N=6。整个编码器堆叠的最终输出被馈送到同样N次堆叠的解码器中。
- 有两个嵌入层、几个跳过连接（每个跳过连接后面跟着一个层归一化层）、几个由两个密集层（第一个使用ReLU激活函数，第二个没有激活函数）组成的前馈模块；输出层是一个使用softmax激活函数的密集层。如果需要的话，在注意力层和前馈模块之后还可以添加一些dropout。由于所有这些层都是时间分布式的（对序列中每个时间步（token）单独、逐个地应用同一个神经网络层。） 词嵌入层 / 前馈网络（FFN）/ Dense 层 本身都是位置无关的，它们不理解顺序。它们就像在每个词上套一层同样的小网络， 所以每个词都是单独处理的，但是，如何通过完全分离的单词来翻译一句话呢？做不到，所以新的组件就派上用场了：

1. 编码器的多头注意力层通过关注同一句子中的所有其他单词来更新每个单词的表示。这就是like这个词的模糊表示变成更丰富、更准确的表示的地方，它变得能够捕捉在给定句子中的精确含义
2. 解码器的掩码多头注意力层也做同样的事情，但处理单词时并不关注其后面的单词：这是一种因果关系层。例如，当它处理单词gusta时，它只会关注“＜SOS＞me gusta”，而忽略“el fútbol”， 提前看到后面的翻译词预测没有意义
3. 解码器的上部多头注意力层是解码器关注英语句子中的单词的地方，这称为交叉注意力，在这种情况下不是自注意力。例如，当解码器处理单词el并将其表示转换为单词fútbol的表示时，它可能会特别关注单词soccer。
4. 位置编码是密集向量（类似于词嵌入），表示句子中每个单词的位置。第n个位置编码被添加到每个句子中的第n个单词的词嵌入中。这是必要的，因为Transformer架构中的所有层都忽略单词位置：没有位置编码，可以随意打乱输入序列，它将以同样的方式打乱输出序列。显然，单词的顺序很重要，这就是我们需要以某种方式给Transformer提供位置信息的原因：将位置编码添加到单词表示中便是实现这一点的好方法。

进入每个多头注意力层的前两个箭头表示键和值，第三个箭头表示查询。在自注意力层中，三者都等于前一层输出的单词表示；而在解码器的上部注意力层中，键和值等于编码器的最终单词表示，查询等于前一层输出的单词表示。

## 位置编码

位置编码是一个密集向量，它对句子中单词的位置进行编码：第i个位置编码被添加到句子中第i个单词的词嵌入中。实现这一点最简单的方法是使用Embedding层并使其对批次中从0到最大序列长度的所有位置进行编码，然后将结果添加到词嵌入中。广播规则将确保位置编码应用于每个输入序列。以下代码展示了如何向编码器和解码器输入添加位置编码：

In [14]:
max_length = 50 # 整个训练集的最大长度
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

请注意，此实现假设嵌入表示为普通张量，而不是不规则张量。编码器和解码器针对位置编码共享相同的Embedding层，因为它们具有相同的嵌入大小（通常都这样）

Transformer论文的作者没有使用可训练的位置编码，而是选择使用基于不同频率的正弦和余弦函数的固定位置编码。位置编码矩阵P在以下公式定义，其中Pp，i是位于句子中第p个位置的单词的编码的第i个分量。

$$
P_{p,i} =
\begin{cases}
\sin\!\left( \dfrac{p}{10000^{i/d}} \right), & i \text{ 是偶数} \\[8pt]
\cos\!\left( \dfrac{p}{10000^{(i-1)/d}} \right), & i \text{ 是奇数}
\end{cases}
$$

该解决方案可以提供与可训练位置编码相同的性能，并且它可以扩展到任意长的句子而无须向模型添加任何参数。在将这些位置编码添加到词嵌入后，模型的其余部分可以访问句子中每个单词的绝对位置，因为每个位置都有一个唯一的位置编码。此外，振荡函数（正弦和余弦）的选择使模型也可以学习相对位置。 因为对于任意一个固定的偏移量 (k)，位置 (pos+k) 的位置编码都可以表示为位置 (pos) 的位置编码的一个线性函数。

为什么每个位置有唯一的位置编码？

多频率 + sin+cos 成对 + 不同周期组合 共同使绝对位置编码在实际序列长度范围内几乎唯一

$$
PE(pos) = [\sin(w_1 pos),\ \cos(w_1 pos),\ \ldots,\ \sin(w_n pos),\ \cos(w_n pos)]
$$


相对位置表示的数学证明：
$$
PE(pos, 2i) = \sin\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

$$
PE(pos, 2i+1) = \cos\left(\frac{pos}{10000^{\frac{2i}{d}}}\right)
$$

$$
\sin(w(pos + k)) = \sin(wpos)\cos(wk) + \cos(wpos)\sin(wk)
$$

$$
\cos(w(pos + k)) = \cos(wpos)\cos(wk) - \sin(wpos)\sin(wk)
$$

$$
\begin{bmatrix}
\sin(w(pos+k)) \\
\cos(w(pos+k))
\end{bmatrix}
=
\begin{bmatrix}
\cos(wk) & \sin(wk) \\
-\sin(wk) & \cos(wk)
\end{bmatrix}
\cdot
\begin{bmatrix}
\sin(wpos) \\
\cos(wpos)
\end{bmatrix}
$$

$$
PE(pos+k) = R_k \cdot PE(pos)
$$






In [15]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length),
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True # keras标记，把输入的mask（掩码/遮罩）传递给输出

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]


pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

## Transformer模型的核心：多头注意力层

**公式 ：缩放点积注意力**

$$
\text{Attention}(Q, K, V)
= \text{softmax}\left(\frac{QK^{\top}}{\sqrt{d_{\text{keys}}}}\right)V
$$

在此等式中：

- **Q** 是每个查询一行的矩阵，其形状为 $(n_{\text{queries}},\, d_{\text{keys}})$。
  $n_{\text{queries}}$ 是查询的数量，而 $d_{\text{keys}}$ 是每个查询和每个键的维度。

- **K** 是每个键一行的矩阵，其形状为 $(n_{\text{keys}},\, d_{\text{keys}})$。
  $n_{\text{keys}}$ 是键和潜在匹配项的数量。

- **V** 是每个值一行的矩阵，其形状为 $(n_{\text{keys}},\, d_{\text{values}})$。
  $d_{\text{values}}$ 是每个值的维度。

- $QK^{\top}$ 的形状是 $(n_{\text{queries}},\, n_{\text{keys}})$：每一对查询/键都会有一个相似性分数。为了防止该矩阵变得太大，输入序列不能太长。softmax 的输出具有相同的形状 $(n_{\text{queries}},\, n_{\text{keys}})$，每一行和为 1。最终输出的形状为 $(n_{\text{queries}},\, d_{\text{values}})$。
每个查询对应一行，其中每行代表查询结果（值的加权和）。


Q/K/V 的本质 = 对输入序列的 embedding 做 3 个不同方向的线性变换。对于每个 token 的 embedding $x\in\mathbb{R}^{d_{\text{model}}}$，Transformer 使用三个线性变换 $W_Q,\ W_K,\ W_V$ 将其投影为 $Q = XW_Q,\ K = XW_K,\ V = XW_V$。

Q 负责“提问”：我要找什么 （搜索关键词）

K 负责“提供线索”：我是什么 （书名/标签，用来被检索）

V 负责“提供内容”：被注意后要拿出的值 （书的正文内容）

- 比例因子 $1/\sqrt{d_{\text{keys}}}$ 会缩小相似性分数，以避免 softmax 函数饱和，这会导致很小的梯度。

- 可以在计算 softmax 之前，通过将一个非常大的负值加到对应的相似性分数中来屏蔽一些键/值。这在掩码多头注意力层中很有用。

如果在创建 `tf.keras.layers.Attention` 层时设置 `use_scale=True`，那么它将创建一个附加参数，让该层学习如何正确缩小相似性分数。Transformer 模型中使用的缩放点积注意力几乎相同，除了它是按输入的因子 $1/\sqrt{d_{\text{keys}}}$ 缩小相似性分数。

请注意，Attention 层的输入 $Q、K 和 V$ 一样，只是多了一个批次维度（第一维度）。在内部，该层只需调用一次
`tf.matmul(queries, keys)` 即可计算出每个子向量的所有注意力分数：这些计算非常高效。
实际上，在 TensorFlow 中，如果 A 和 B 是具有两个以上维度的张量——例如 A 的形状分别为 `[2, 3, 4, 5]` 和 B 为 `[2, 3, 5, 6]`，
那么 `tf.matmul(A, B)` 将把这些矩阵视为 $2 \times 3$ 组矩阵（其中每个单元代表一个矩阵），并将它们的矩阵相乘：
A 中第 $i$ 行的矩阵将与 B 中第 $i$ 行的矩阵相乘。
由于 $4 \times 5$ 的矩阵乘以 $5 \times 6$ 的矩阵结果是 $4 \times 6$，因此 `tf.matmul(A, B)` 将返回形状为 `[2, 3, 4, 6]` 的数组。



多头注意力层，其架构如图所示

![多头注意力层架构](./images/transformer/p2.png)

它只是一堆缩放点积注意力层，可以看成做了多次缩放点积注意力，每个层之前都有值、键和查询的线性变换（即没有激活函数的时间分布密集层）。所有的输出都被简单地连接起来，并经过最终线性变换（同样是时间分布的）。

$$
\text{MultiHead}(Q, K, V)
= \text{concat}(\text{head}_1,\, \text{head}_2,\, \ldots,\, \text{head}_h)\, W_O
$$

$$
\text{head}_i
= \text{Attention}\left(W_i^{Q} Q,\; W_i^{K} K,\; W_i^{V} V\right)
$$




这种架构背后的直觉是什么？再考虑一下“I like soccer”这句话中的like。编码器足够智能地编码了它是动词的事实。但是由于位置编码，单词表示还包括它在文本中的位置，并且它可能包括许多对其翻译有用的其他特征，例如它是现在时的事实。简而言之，单词表示编码了单词的许多不同特征。如果我们只使用一个单一的缩放点积注意力层，那么只能一次查询所有这些特征。这就是多头注意力层对值、键和查询应用多种不同线性变换的原因：这允许模型将单词表示的许多不同投影应用到不同的子空间，每个子空间都关注单词的一个特征子集。也许其中一个线性层会将单词表示投影到一个子空间中，其中剩下的只是单词是动词的信息，另一个线性层将仅提取它是现在时的事实，等等。然后，缩放点积注意力层实现查找阶段，最后我们连接所有结果并将它们投射回原始空间。Keras包含一个tf.keras.layers.MultiHeadAttention层，因此现在拥有构建Transformer所需的一切

从完整的编码器开始，它与图中的架构完全一样，除了使用2个而不是6个块堆叠(N=2)，同时添加了一些dropout做正则化：

In [ ]:
# 输入 X: (batch, seq_len, d_model)
#    ↓ 线性变换 W_Q/W_K/W_V
# Q,K,V: (batch, seq_len, d_model)
#    ↓ reshape split
# Q,K,V: (batch, h, seq_len, d_k)
#    ↓ 对每个头注意力
# head_i: (batch, seq_len, d_k)
#    ↓ concat
# concat: (batch, seq_len, h*d_k = d_model)
#    ↓ 线性变换 W_O
# output: (batch, seq_len, d_model)

In [16]:
N = 2 # 论文里是6
num_heads = 8
dropout_rate = 0.1

n_units = 128  # 前馈网络的第一个全连接层
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

着重讲下这段代码的掩码部分：

MultiHeadAttention 层接受一个 `attention_mask` 参数，它是形状为$ \text{（批量大小，最大查询长度，最大键长度）} $ 的布尔张量：对于每个查询序列中的每个词元，此掩码指示相应序列中的哪些词元应该被关注。我们想告诉 MultiHeadAttention 层忽略值中的所有填充词元。因此，我们首先使用
`tf.math.not_equal(encoder_input_ids, 0)` 来计算填充掩码。这将返回形状为 $ \text{（批量大小，最大序列长度）} $ 的布尔张量。

然后，我们使用[:, tf.newaxis]插入第二个轴，以获得形状  $ \text{（批量大小，1，最大序列长度）} $ 的掩码。这样我们在调用 MultiHeadAttention 层时便可将此掩码作为 `attention_mask`：  由于广播，将相同的掩码用于每个查询中的所有行。这样，值中的填充词元将被正确忽略。

但是，该层将为每个查询词元（包括填充词元）计算输出。  需要屏蔽对应于这些填充词元的输出。  在 Embedding 层中使用了 `mask_zero`，并且在 PositionalEncoding 层中将  `supports_masking=True`，因此自动掩码一直会传播到 MultiHeadAttention 层的输入（`encoder_in`）。

可以在skip连接中利用这一点：事实是，Add 层支持自动掩码，所以当我们添加 add 或 skip（最终等于 encoder_in）时，输出会自动正确地被屏蔽。



解码器, 掩码也是唯一棘手的部分，所以从它开始。第一个多头注意力层是一个自注意力层，就像在编码器中一样，但它是一个掩码的多头注意力层，这意味着它是因果关系层：它应该忽略未来所有词元。所以，需要两个掩码：一个填充掩码和一个因果掩码。来创建它们：



In [17]:
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(  # 创建下三角矩阵
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

填充掩码与为编码器创建的掩码完全一样，只是它基于解码器的输入而不是编码器的输入。因果掩码是使用tf.linalg.band_part()函数创建的，该函数接受一个张量并返回一个副本（其中对角线带外的所有值都设置为零）。使用这些参数，得到一个大小为batch_max_len_dec（批次中输入序列的最大长度）的方阵，左下三角元素均为1，右上三角元素均为0。如果我们使用这个掩码作为注意力掩码，我们将得到我们想要的：第一个查询词元将只关注第一个值词元，第二个只关注前两个，第三个只关注前三个，以此类推。换句话说，查询词元不能未来的任何值词元。

In [18]:
# 构建解码器
encoder_outputs = Z  # let's save the encoder's final outputs
Z = decoder_in  # the decoder starts with its own inputs
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

对于第一个注意力层，使用causal_mask & decoder_pad_mask来屏蔽填充词元和未来词元。因果掩码只有两个维度：它缺少批次维度，但这没关系，因为广播可以确保它被复制到批次中的所有实例。

第二个注意力层没有什么特别的。唯一需要注意的是我们使用的是encoder_pad_mask，而不是decoder_pad_mask，因为这个注意力层使用编码器的最终输出作为它的值。

In [19]:
Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 1660s 529ms/step - loss: 2.7615 - accuracy: 0.4524 - val_loss: 1.9346 - val_accuracy: 0.5737
Epoch 2/10
3125/3125 [==============================] - 1440s 461ms/step - loss: 1.7994 - accuracy: 0.5936 - val_loss: 1.5986 - val_accuracy: 0.6312
Epoch 3/10
3125/3125 [==============================] - 1405s 450ms/step - loss: 1.5756 - accuracy: 0.6327 - val_loss: 1.4711 - val_accuracy: 0.6553
Epoch 4/10
3125/3125 [==============================] - 1405s 450ms/step - loss: 1.4626 - accuracy: 0.6527 - val_loss: 1.3912 - val_accuracy: 0.6700
Epoch 5/10
3125/3125 [==============================] - 1402s 449ms/step - loss: 1.3885 - accuracy: 0.6656 - val_loss: 1.3403 - val_accuracy: 0.6802
Epoch 6/10
3125/3125 [==============================] - 1402s 449ms/step - loss: 1.3370 - accuracy: 0.6744 - val_loss: 1.3114 - val_accuracy: 0.6851
Epoch 7/10
3125/3125 [==============================] - 1404s 449ms/step - loss: 1.2969 - accuracy: 0.6814

In [22]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 22ms/step


'me gusta el fútbol y también va a la playa'